In [ ]:
!pip install tensorflow
!pip install numpy pillow

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# load image as tf
def load_img(path_to_img):
    max_dim = 500
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

Load and display images

In [ ]:
content_path = 'cat.jpg'
style_path = 'starry_night.jpg'

content_img = load_img(content_path)
style_img = load_img(style_path)

In [ ]:
plt.imshow(content_img[0])
print(content_img.shape)
plt.show()

plt.imshow(style_img[0])
print(style_img.shape)
plt.show()

Get layers from pretrained VGG model

In [ ]:
def get_vgg_layers(style_layers_names, content_layers_names):
    # pretrained vgg cnn
    vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet', pooling='avg')
    vgg.trainable = False
    model_layers_names = style_layers_names + content_layers_names
    model_outputs = [vgg.get_layer(name).output for name in model_layers_names]
    return tf.keras.Model([vgg.input], model_outputs)

In [ ]:
style_layers_names = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
content_layers_names = ['block1_conv2', 'block2_conv2', 'block3_conv2']
num_content_layers = len(content_layers_names)
num_style_layers = len(style_layers_names)


In [ ]:
class ArtisticCNN(tf.keras.models.Model):
  def __init__(self, style_layers_names, content_layers_names):
    super(ArtisticCNN, self).__init__()
    self.vgg = get_vgg_layers(style_layers_names, content_layers_names)
    self.style_layers = style_layers_names
    self.content_layers = content_layers_names
    self.vgg.trainable = False

  def call(self, inputs):
    # make sure inputs are floats in [0,1]
    # returns style outputs then content outputs
    inputs = inputs * 255.0

    input = tf.keras.applications.vgg19.preprocess_input(inputs)
    all_outputs = self.vgg(input)

    style_outputs = all_outputs[:num_style_layers]
    style_outputs = [gram_matrix(style_output) for style_output in style_outputs]
    content_outputs = all_outputs[num_style_layers:]

    return style_outputs, content_outputs

In [ ]:
model = ArtisticCNN(style_layers_names, content_layers_names)
results = model(tf.constant(content_img))

style_targets, _ = model(style_img)
_, content_targets = model(content_img)

In [ ]:
image = tf.Variable(load_img(content_path)) #initialize with content photo

epochs = 20
steps_per_epoch = 50

In [ ]:
# calculate gram matrix
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    num_locations = tf.cast(n, tf.float32)

    return gram / num_locations

In [ ]:
def get_total_loss(style_outputs, content_outputs):

    style_loss = tf.add_n([tf.reduce_mean((style_outputs[i]-style_targets[i])**2) for i in range(num_style_layers)])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[i]-content_targets[i])**2) for i in range(num_content_layers)])
    content_loss *= content_weight / num_content_layers

    loss = style_loss + content_loss
    return loss

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.2, beta_1=0.99, epsilon=1e-1)

style_weight=1e-2
content_weight=1e3


In [ ]:
image = tf.Variable(content_img) #initialize with content photo

epochs = 20
steps_per_epoch = 50

In [ ]:
step = 0
for n in range(epochs):
  plt.imshow(np.squeeze(image.read_value(), 0))
  plt.title("Training epoch: {}".format(n))
  plt.savefig('cat_epoch'+str(n)+'.jpeg', bbox_inches='tight')
  plt.show()

  for m in range(steps_per_epoch):
    step += 1
    with tf.GradientTape() as tape:
      style_outputs, content_outputs = model(image)
      loss = get_total_loss(style_outputs, content_outputs)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    clipped_image = tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)
    image.assign(clipped_image)
